In [1]:
import urllib.request, json 
import pandas as pd
import numpy as np
import math

In [2]:
data = pd.read_csv("clean_csv/akoya.csv")

managers = ["Ali","Ruslan","Sami","Yahya","Youssef","Santi","Shrey","Dani"]

current_gw = 38 #current gameweek

In [3]:
data.head()

,player_id,player_name,position,team,gameweek,manager_id,squad_position,bench,points,minutes,...,saves,bonus,bps,influence,creativity,threat,ict_index,total_points,in_dreamteam,photo
0,1.0,Cédric,DEF,FUL,5.0,transfer market,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,58822
1,2.0,Leno,GK,FUL,5.0,transfer market,NaN,NaN,2.0,90.0,...,2.0,0.0,13.0,19.0,10.0,0.0,2.9,2.0,False,80201
2,105.0,Duffy,DEF,FUL,5.0,transfer market,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,61933
3,200.0,Cairney,MID,FUL,5.0,transfer market,NaN,NaN,1.0,12.0,...,0.0,0.0,4.0,0.0,1.7,4.0,0.6,1.0,False,76357
4,201.0,Ream,DEF,FUL,5.0,transfer market,NaN,NaN,2.0,90.0,...,0.0,0.0,17.0,16.2,0.7,0.0,1.7,2.0,False,82514


In [4]:
data.columns

Index(['player_id', 'player_name', 'position', 'team', 'gameweek',
       'manager_id', 'squad_position', 'bench', 'points', 'minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'total_points', 'in_dreamteam', 'photo'],
      dtype='object')

# Points

In [5]:
gameweek_points = data[(data["manager_id"]!="transfer market")&(data["bench"]=="fielded")]

grouped = gameweek_points.groupby(["manager_id","gameweek"])["points"].sum()
gameweek_df = grouped.reset_index()
gameweek_df

,manager_id,gameweek,points
0,Ali,5.0,51.0
1,Ali,6.0,43.0
2,Ali,7.0,0.0
3,Ali,8.0,54.0
4,Ali,9.0,53.0
...,...,...,...
267,Youssef,34.0,57.0
268,Youssef,35.0,36.0
269,Youssef,36.0,40.0
270,Youssef,37.0,42.0


In [6]:
def get_ranking(df,bench,goal):
    filtered = df[(df["manager_id"]!="transfer market")&(df["bench"]==bench)&(df["gameweek"]!=7)]
    ranked_df = filtered.groupby(df["manager_id"])
    return ranked_df[goal].sum().sort_values(ascending=False)

In [7]:
get_ranking(data,"fielded","points")

manager_id
Ali        1553.0
Sami       1512.0
Ruslan     1479.0
Yahya      1434.0
Youssef    1384.0
Santi      1310.0
Shrey      1273.0
Dani       1226.0
Name: points, dtype: float64

In [8]:
real_ranking = get_ranking(data,"fielded","points").to_frame()
real_ranking = real_ranking.reset_index().rename(columns={'manager_id': 'manager'})
real_ranking.to_csv(f'findings/points/real_ranking.csv', index=False)

In [9]:
real_ranking.columns

Index(['manager', 'points'], dtype='object')

### League by positions

Make 4 tables, ranking everyone depending on each position (gk, def, mid, fwd)

In [10]:
#GK Table
gk = data[data["position"]=="GK"]
get_ranking(gk,"fielded","points")

manager_id
Shrey      154.0
Ruslan     143.0
Youssef    139.0
Dani       137.0
Sami       133.0
Santi      132.0
Ali        127.0
Yahya      109.0
Name: points, dtype: float64

In [11]:
#DEF Table
deef = data[data["position"]=="DEF"]
get_ranking(deef,"fielded","points")

manager_id
Sami       554.0
Ruslan     463.0
Youssef    433.0
Dani       417.0
Ali        411.0
Yahya      372.0
Santi      330.0
Shrey      318.0
Name: points, dtype: float64

In [12]:
#MID Table
mid = data[data["position"]=="MID"]
get_ranking(mid,"fielded","points")

manager_id
Youssef    628.0
Shrey      624.0
Yahya      607.0
Ali        601.0
Ruslan     584.0
Sami       536.0
Santi      504.0
Dani       437.0
Name: points, dtype: float64

In [13]:
#FWD Table
fwd = data[data["position"]=="FWD"]
get_ranking(fwd,"fielded","points")

manager_id
Ali        414.0
Yahya      346.0
Santi      344.0
Ruslan     289.0
Sami       289.0
Dani       235.0
Youssef    184.0
Shrey      177.0
Name: points, dtype: float64

In [14]:
gks_df = get_ranking(gk,"fielded","points").to_frame()
def_df = get_ranking(deef,"fielded","points").to_frame()
mid_df = get_ranking(mid,"fielded","points").to_frame()
fwd_df = get_ranking(fwd,"fielded","points").to_frame()

gks_df = gks_df.reset_index().rename(columns={'manager_id': 'manager'})
def_df = def_df.reset_index().rename(columns={'manager_id': 'manager'})
mid_df = mid_df.reset_index().rename(columns={'manager_id': 'manager'})
fwd_df = fwd_df.reset_index().rename(columns={'manager_id': 'manager'})

gks_df.to_csv(f'findings/points/gk_ranking.csv', index=False)
def_df.to_csv(f'findings/points/def_ranking.csv', index=False)
mid_df.to_csv(f'findings/points/mid_ranking.csv', index=False)
fwd_df.to_csv(f'findings/points/fwd_ranking.csv', index=False)

### Best gw by position

In [15]:
def get_ranking_gw(df):
    filtered = df[(df["manager_id"]!="transfer market")&(df["bench"]=="fielded")&(df["gameweek"]!=7)]
    ranked_df = filtered.groupby(["manager_id","gameweek"])
    return ranked_df["points"].sum().sort_values(ascending=False)

In [16]:
#GK GW Table
get_ranking_gw(gk)

manager_id  gameweek
Shrey       12.0        15.0
Sami        34.0        15.0
Ali         29.0        14.0
Sami        19.0        14.0
Shrey       11.0        14.0
                        ... 
Youssef     8.0          0.0
Dani        30.0         0.0
Youssef     26.0        -1.0
Sami        35.0        -2.0
Ali         24.0        -3.0
Name: points, Length: 264, dtype: float64

In [17]:
#DEF GW Table
get_ranking_gw(deef)

manager_id  gameweek
Sami        36.0        48.0
Ruslan      25.0        47.0
            27.0        41.0
Youssef     25.0        36.0
Sami        29.0        35.0
                        ... 
Yahya       26.0         2.0
Santi       9.0          1.0
Shrey       28.0         1.0
            32.0         0.0
Santi       8.0         -3.0
Name: points, Length: 264, dtype: float64

In [18]:
#MID GW Table
get_ranking_gw(mid)

manager_id  gameweek
Ali         29.0        44.0
Shrey       20.0        42.0
            9.0         41.0
Youssef     14.0        40.0
            23.0        40.0
                        ... 
Santi       12.0         4.0
Youssef     38.0         4.0
Ali         32.0         3.0
Dani        25.0         2.0
Ali         28.0         2.0
Name: points, Length: 264, dtype: float64

In [19]:
#FWD GW Table
get_ranking_gw(fwd)

manager_id  gameweek
Dani        37.0        30.0
Santi       36.0        30.0
Yahya       38.0        29.0
Santi       34.0        28.0
Ali         9.0         25.0
                        ... 
Sami        6.0          1.0
Ruslan      37.0         1.0
Dani        25.0         0.0
Santi       8.0          0.0
Ruslan      19.0         0.0
Name: points, Length: 264, dtype: float64

In [20]:
gks_df = get_ranking_gw(gk).to_frame()
def_df = get_ranking_gw(deef).to_frame()
mid_df = get_ranking_gw(mid).to_frame()
fwd_df = get_ranking_gw(fwd).to_frame()

gks_df = gks_df.reset_index().rename(columns={'manager_id': 'manager'})
def_df = def_df.reset_index().rename(columns={'manager_id': 'manager'})
mid_df = mid_df.reset_index().rename(columns={'manager_id': 'manager'})
fwd_df = fwd_df.reset_index().rename(columns={'manager_id': 'manager'})

gks_df.to_csv(f'findings/points/gw_gk_ranking.csv', index=False)
def_df.to_csv(f'findings/points/gw_def_ranking.csv', index=False)
mid_df.to_csv(f'findings/points/gw_mid_ranking.csv', index=False)
fwd_df.to_csv(f'findings/points/gw_fwd_ranking.csv', index=False)

### Bench FC


In [21]:
get_ranking(data,"bench","points")

manager_id
Ruslan     269.0
Sami       251.0
Santi      239.0
Ali        225.0
Dani       207.0
Shrey      157.0
Youssef    157.0
Yahya      145.0
Name: points, dtype: float64

In [22]:
bench = get_ranking(data,"bench","points").to_frame()

bench = bench.reset_index().rename(columns={'manager_id': 'manager'})

bench_total = bench.groupby("manager").sum().sort_values("points",ascending=False)
bench_total = bench_total.reset_index()

bench.to_csv(f'findings/points/bench.csv', index=False)

### Optimised Bench

In [23]:
def optimise(gw,manager):
    benched = data[(data["gameweek"] == gw)&(data["manager_id"] == manager)&(data["bench"] == "bench")]
    fielded = data[(data["gameweek"] == gw)&(data["manager_id"] == manager)&(data["bench"] == "fielded")]

    points = 0

    positions = [("DEF",3),("MID",2),("FWD",1)]

    b_gk = benched[benched["position"] == "GK"]
    f_gk = fielded[fielded["position"] == "GK"]

    if  f_gk.loc[f_gk.index.tolist()[0],"points"] < b_gk.loc[b_gk.index.tolist()[0],"points"]:
        points += b_gk["points"].item() - f_gk["points"].item()
        benched = benched.drop(index=b_gk.index)

    for indx, b_player in benched.iterrows():
        for pos in positions:
            for indx2, f_player in fielded.iterrows():
                if len(fielded[fielded["position"] == pos[0]]) > pos[1]:
                    if f_player["points"] < b_player["points"]:
                        points += b_player["points"] - f_player["points"]
                        benched = benched.drop(index=indx)
                        fielded = fielded.drop(index=indx2)
                        break  # exit the inner loop and go to the next b_player
            else:
                continue  # only executed if the inner loop didn't break
            break  # exit the outer loop and go to the next b_player

    return points

In [24]:
missed_points = pd.DataFrame(columns=["manager","gameweek","missed_pts"])

for manager in managers:
    for gw in range(5,current_gw):
        new_row = {"manager":manager,"gameweek":gw,"missed_pts":optimise(gw,manager)}
        missed_points = missed_points.append(new_row, ignore_index=True)


C:\Users\santi\AppData\Local\Temp\ipykernel_24432\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_points = missed_points.append(new_row, ignore_index=True)
C:\Users\santi\AppData\Local\Temp\ipykernel_24432\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_points = missed_points.append(new_row, ignore_index=True)
C:\Users\santi\AppData\Local\Temp\ipykernel_24432\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_points = missed_points.append(new_row, ignore_index=True)
C:\Users\santi\AppData\Local\Temp\ipykernel_24432\1275510679.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

In [25]:
missed_points

,manager,gameweek,missed_pts
0,Ali,5,1.0
1,Ali,6,5.0
2,Ali,7,0.0
3,Ali,8,2.0
4,Ali,9,0.0
...,...,...,...
259,Dani,33,0.0
260,Dani,34,0.0
261,Dani,35,0.0
262,Dani,36,0.0


In [26]:
missed_points_grouped = missed_points.groupby(missed_points["manager"])
missed_points_grouped["missed_pts"].sum().sort_values(ascending=False)

manager
Sami       158.0
Ruslan     148.0
Ali        137.0
Santi      136.0
Dani       110.0
Shrey      101.0
Youssef     98.0
Yahya       83.0
Name: missed_pts, dtype: float64

In [27]:
bench_best = missed_points_grouped["missed_pts"].sum().sort_values(ascending=False).to_frame()
bench_best = bench_best.reset_index().rename(columns={'missed_pts': 'points'})
bench_best.to_csv(f'findings/points/bench_best.csv', index=False)

### GW Podiums

In [28]:
gw_podiums = pd.DataFrame(columns=["manager","gameweek","points"])

gw_podiums["manager"] = gameweek_df["manager_id"]
gw_podiums["gameweek"] = gameweek_df["gameweek"]
gw_podiums["points"] = gameweek_df["points"]
gw_podiums["rank"] = gw_podiums.groupby("gameweek")["points"].rank(ascending=False).apply(math.floor)
gw_podiums["podium"] = gw_podiums["rank"]<4


gw_podiums

,manager,gameweek,points,rank,podium
0,Ali,5.0,51.0,1,True
1,Ali,6.0,43.0,2,True
2,Ali,7.0,0.0,4,False
3,Ali,8.0,54.0,1,True
4,Ali,9.0,53.0,2,True
...,...,...,...,...,...
267,Youssef,34.0,57.0,4,False
268,Youssef,35.0,36.0,4,False
269,Youssef,36.0,40.0,7,False
270,Youssef,37.0,42.0,5,False


In [29]:
gw_podiums[gw_podiums["manager"] == "Youssef"]
gw_podiums.groupby("manager")["rank"].mean().sort_values()

manager
Ali        3.470588
Sami       3.941176
Yahya      4.058824
Ruslan     4.441176
Youssef    4.470588
Santi      4.823529
Shrey      5.088235
Dani       5.352941
Name: rank, dtype: float64

In [30]:
total_podiums_df = pd.DataFrame(index=['1st', '2nd', '3rd'], columns=gw_podiums['manager'].unique())

grouped = gw_podiums.groupby(['manager', 'rank']).size()

# loop through each manager and count their 1st, 2nd, and 3rd place finishes
for manager in total_podiums_df.columns:
    try:
        total_podiums_df.loc['1st', manager] = grouped[manager, 1]
    except KeyError:
        total_podiums_df.loc['1st', manager] = 0
    try:
        total_podiums_df.loc['2nd', manager] = grouped[manager, 2]
    except KeyError:
        total_podiums_df.loc['2nd', manager] = 0
    try:
        total_podiums_df.loc['3rd', manager] = grouped[manager, 3]
    except KeyError:
        total_podiums_df.loc['3rd', manager] = 0

total_podiums_df.loc['Total'] = total_podiums_df.sum(axis=0)

total_podiums_df

,Ali,Dani,Ruslan,Sami,Santi,Shrey,Yahya,Youssef
1st,8,2,8,8,2,2,3,0
2nd,6,2,2,2,4,4,8,6
3rd,5,4,5,4,6,2,3,6
Total,19,8,15,14,12,8,14,12


In [31]:
total_podiums_df = total_podiums_df.transpose()

In [32]:
total_podiums_df.to_csv(f'findings/points/podiums.csv', index=True,index_label="manager")

### Tottenham Award

Most GW without podium

In [33]:
def longest_streak(df,who):
    current_streak = 1
    max_streak = 1
    indx = 0
    
    df = df[df["manager"]==who]
    
    series = df["podium"]

    for i in range(1, len(series)):
        if series.iloc[i]:
            max_streak = max(max_streak, current_streak)
            current_streak = 1
        else:
            current_streak += 1
            if current_streak > max_streak:
                indx = i

    return max(max_streak, current_streak),df.iloc[indx]["manager"]

In [34]:
longest_nopodium_streak = (0,"")
tottenham = pd.DataFrame(columns=["streak_length"])

for manager in managers:
    streak = longest_streak(gw_podiums, manager)
    if longest_nopodium_streak[0] < streak[0]:
        longest_nopodium_streak = streak
    tottenham.loc[manager,"streak_length"] = streak[0]
    

longest_nopodium_streak

(15, 'Dani')

In [35]:
tottenham = tottenham.sort_values(by="streak_length",ascending=False)

In [36]:
tottenham = tottenham.reset_index().rename(columns={'index': 'manager'})

tottenham.to_csv(f'findings/points/tottenham.csv', index=False)

In [37]:
tottenham

,manager,streak_length
0,Dani,15
1,Shrey,13
2,Youssef,9
3,Sami,7
4,Santi,7
5,Ruslan,5
6,Ali,4
7,Yahya,4


### GW Losers

In [38]:
last_df = pd.DataFrame(index=["Last"], columns=gw_podiums['manager'].unique())

last_grouped = gw_podiums.groupby(['manager', 'rank']).size()

# loop through each manager and count their 1st, 2nd, and 3rd place finishes
for manager in last_df.columns:
    try:
        last_df.loc['Last', manager] = last_grouped[manager, 8]
    except KeyError:
        last_df.loc['1st', manager] = 0


last_df = last_df.transpose().sort_values(by="Last",ascending=False)

In [39]:
last_df = last_df.reset_index().rename(columns={'index': 'manager'})

last_df.to_csv(f'findings/points/last_df.csv', index=False)

# Players

### 1-team Players

In [40]:
def get_loyalty(data):
    grouped = data.groupby(['manager_id',"player_name","team","photo"]).agg({'player_id': 'size', 'points': 'sum'}).sort_values('player_id',ascending=False)
    return grouped

In [41]:
#most owned players per club
no_tm = data[(data["manager_id"]!="transfer market")]
loyalty_df = get_loyalty(no_tm)
loyalty_df=loyalty_df.drop(columns="points")
loyalty_df

player_id
manager_id player_name   team photo            
Yahya      Coady         EVE  94147          34
           Calvert-Lewin EVE  177815         34
           Kane          TOT  78830          34
Youssef    Dias          MCI  171314         34
           De Gea        MUN  51940          34
...                                         ...
Sami       March         BHA  109345          1
           Nørgaard      BRE  128295          1
           Ouattara      BOU  533463          1
           Pedro Porro   TOT  441164          1
           Murphy        NEW  114243          1

[447 rows x 1 columns]

In [42]:
loyalty_df.loc["Youssef"]

,,,player_id
player_name,team,photo,
Dias,MCI,171314,34
De Gea,MUN,51940,34
De Bruyne,MCI,61366,34
Martinelli,ARS,444145,34
Van Dijk,LIV,97032,34
Schär,NEW,119471,32
Almirón,NEW,179018,29
Johnson,NFO,242898,27
Havertz,CHE,219847,22


In [43]:
#most fielded players per club
most_played_df = get_loyalty(gameweek_points)
most_played_df

player_id  points
manager_id player_name team photo                    
Yahya      Kane        TOT  78830          34   237.0
           Saka        ARS  223340         33   186.0
Shrey      Alisson     LIV  116535         33   152.0
Yahya      Ederson     MCI  121160         33   106.0
Shrey      Fernandes   MUN  141746         33   164.0
...                                       ...     ...
Santi      Webster     BHA  110735          1     0.0
Ruslan     Watkins     AVL  178301          1     7.0
Santi      Toney       BRE  144485          1     9.0
           Tete        FUL  167074          1     1.0
Ali        A.Armstrong SOU  155511          1     2.0

[412 rows x 2 columns]

In [44]:
loyalty = loyalty_df.reset_index().rename(columns={'manager_id': 'manager'})
most_played = most_played_df.reset_index().rename(columns={'manager_id': 'manager'})
most_played["ppg"] = round(most_played["points"]/most_played["player_id"],2)

loyalty.to_csv(f'findings/players/loyalty.csv', index=False)
most_played.to_csv(f'findings/players/most_played.csv', index=False)

Players in the most teams

In [45]:
most_teams_grouped = loyalty_df.groupby(["player_name","team","photo"]).size().sort_values(axis=0,ascending=False)
most_teams_df = most_teams_grouped.to_frame().sort_values(by=[0, 'player_name'], ascending=[False, True])
most_teams_df[most_teams_df.iloc[:,0]>3]

,,,0
player_name,team,photo,
Chilwell,CHE,172850,5
Sancho,MUN,209243,5
Edouard,CRY,199670,4
Eriksen,MUN,80607,4
Gündogan,MCI,59859,4
Jensen,BRE,207283,4
Martial,MUN,148225,4
Martínez,MUN,221820,4
Walker,MCI,58621,4


In [46]:
most_teams_df.columns

RangeIndex(start=0, stop=1, step=1)

In [47]:
most_teams_df = most_teams_df.reset_index()

most_teams_df.to_csv(f'findings/players/most_teams.csv', index=False)

### Club Mascot

In [48]:
def get_clubmascot(data):
    mascot_grouped = data.groupby(['manager_id',"team"]).sum().sort_values(by=["player_id", 'manager_id'], ascending=[False, True])
    return mascot_grouped

In [49]:
#most owned club
club_mascot = get_clubmascot(most_played_df)
club_mascot["bench"] = get_clubmascot(loyalty_df)
club_mascot.rename(columns={"player_id": 'fielded'}, inplace=True)
club_mascot["ratio"] = round(club_mascot["fielded"]/club_mascot["bench"],2)
club_mascot["ppg"] = round(club_mascot["points"]/club_mascot["fielded"],2)
club_mascot[club_mascot["fielded"]>15].sort_values(by="points",ascending=False)[:20]

fielded  points  bench  ratio   ppg
manager_id team                                     
Ruslan     ARS       202   878.0    243   0.83  4.35
Ali        MCI       101   502.0    126   0.80  4.97
Sami       LIV        71   379.0     86   0.83  5.34
Ali        MUN        67   339.0     86   0.78  5.06
Dani       BRE        92   308.0    107   0.86  3.35
Shrey      MCI        64   257.0     90   0.71  4.02
           MUN        57   250.0     63   0.90  4.39
Sami       NEW        53   238.0     60   0.88  4.49
Yahya      TOT        34   237.0     34   1.00  6.97
Youssef    NEW        53   230.0     61   0.87  4.34
           MCI        68   225.0     89   0.76  3.31
Sami       BHA        41   209.0     45   0.91  5.10
Santi      AVL        39   187.0     44   0.89  4.79
Yahya      ARS        34   186.0     41   0.83  5.47
           CHE        58   179.0     77   0.75  3.09
Youssef    ARS        37   178.0     46   0.80  4.81
           BHA        32   171.0     41   0.78  5.34
Shrey      TOT        47   167.0     95   0.49  3.55
           LIV        43   161.0     53   0.81  3.74
Santi      NEW        32   161.0     47   0.68  5.03

In [50]:
club_mascot = club_mascot.reset_index().rename(columns={'manager_id': 'manager'})

club_mascot.to_csv(f'findings/players/club_mascot.csv', index=False)

# Stats

### Most Goals

In [51]:
stats = get_ranking(data,"fielded","goals_scored").to_frame()
bench_stats = get_ranking(data,"bench","goals_scored").to_frame()

### Most Assists

In [52]:
assists = get_ranking(data,"fielded","assists").to_frame()
assists_bench = get_ranking(data,"bench","assists").to_frame()
stats["assists"] = assists["assists"]
bench_stats["assists_bench"] = assists_bench["assists"]

### Most Clean Sheets

In [53]:
gk_def = data[(data["position"]=="GK")|(data["position"]=="DEF")]

clean_sheets = get_ranking(gk_def,"fielded","clean_sheets").to_frame()
clean_sheets_bench = get_ranking(gk_def,"bench","clean_sheets").to_frame()
stats["clean_sheets"] = clean_sheets["clean_sheets"]
bench_stats["clean_sheets_bench"] = clean_sheets_bench["clean_sheets"]

### Avg Minutes Played

In [54]:
minutes = round(get_ranking(data,"fielded","minutes").to_frame()/11/31)
stats["minutes"] = minutes["minutes"]

### Maguire Award
Most Goals Conceded

In [55]:
goals_conceded = get_ranking(gk_def,"fielded","goals_conceded").to_frame()
goals_conceded_bench = get_ranking(gk_def,"bench","goals_conceded").to_frame()
stats["goals_conceded"] = goals_conceded["goals_conceded"]
bench_stats["goals_conceded_bench"] = goals_conceded_bench["goals_conceded"]

### 2016 Pessi Award
Most Missed Pens

In [56]:
penalties = get_ranking(data,"fielded","penalties_missed").to_frame()
saved_penalties = get_ranking(data,"fielded","penalties_saved").to_frame()
stats["penalties_missed"] = penalties["penalties_missed"]
stats["penalties_saved"] = saved_penalties["penalties_saved"]

### Not De Gea Award
Most Pens Saved

In [57]:
get_ranking(data,"fielded","penalties_saved")

manager_id
Sami       2.0
Ali        1.0
Dani       1.0
Santi      1.0
Shrey      1.0
Youssef    1.0
Ruslan     0.0
Yahya      0.0
Name: penalties_saved, dtype: float64

### Prime Maguire Award
Most Own Goals

In [58]:
own_goals = get_ranking(data,"fielded","own_goals").to_frame()
stats["own_goals"] = own_goals["own_goals"]

### Sergio Ramos Award
Most Red Cards

In [59]:
red_cards = get_ranking(data,"fielded","red_cards").to_frame()
stats["red_cards"] = red_cards["red_cards"]

### Sergio Ramos Lite Award
Most Yellow Cards

In [60]:
yellow_cards = get_ranking(data,"fielded","yellow_cards").to_frame()
stats["yellow_cards"] = yellow_cards["yellow_cards"]

### The Wall Award
Most Saves

In [61]:
saves = get_ranking(data,"fielded","saves").to_frame()
stats["saves"] = saves["saves"]

### BPS Merchant

In [62]:
filtered_bps = data[(data["manager_id"]!="transfer market")&(data["bench"]=="fielded")]
df = filtered_bps.groupby(data["manager_id"])
bonus = df["bonus"].sum().sort_values(ascending=False).to_frame()
stats["bonus"] = bonus["bonus"]

### TOTW Merchant

In [63]:
dreamteam = get_ranking(data,"fielded","in_dreamteam").to_frame()
stats["dreamteam"] = dreamteam["in_dreamteam"]

In [64]:
stats.to_csv("findings/stats/stats.csv")
bench_stats.to_csv("findings/stats/bench_stats.csv")

# Transfers

In [65]:
transfers = pd.read_csv("clean_csv/transfers.csv")

In [66]:
transfers[:10]

,players_id_in,players_id_out,manager_id,gameweek,type,result,transfer_id,player_name_in,photo_in,team_in,player_name_out,photo_out,team_out
0,189,64,Ali,5,f,a,8830907,Gray,172632,EVE,Lerma,152551,BOU
1,346,465,Yahya,5,f,a,8833951,Andreas,156689,FUL,Bowen,178186,WHU
2,134,259,Ruslan,5,f,a,8834214,Kanté,116594,CHE,Tielemans,166989,LEI
3,358,43,Yahya,5,f,a,8838057,Burn,78916,NEW,Cash,199796,AVL
4,342,504,Sami,5,f,a,8857423,Dalot,216051,MUN,Malacia,222690,MUN
5,235,8,Ruslan,5,f,a,8881578,Struijk,222694,LEE,Tierney,192895,ARS
6,344,469,Ali,5,f,a,8918574,Elanga,449434,MUN,Fornals,217593,WHU
7,603,142,Yahya,6,w,di,8969639,Paquetá,224024,WHU,Mount,184341,CHE
8,526,139,Yahya,6,w,a,8969640,Mee,51927,BRE,Chilwell,172850,CHE
9,319,227,Youssef,6,w,a,9210908,Álvarez,461358,MCI,Bamford,106617,LEE


In [67]:
def get_ranking_transfer(df,types):
    if types == "all":
        filtered = df
    else:
        filtered = df[(df["type"]==types)]
    ranked_df = filtered.groupby(df["manager_id"])
    return ranked_df["transfer_id"].size().sort_values(ascending=False)

### Ranking by Total Transfers

In [68]:
accepted = transfers[transfers["result"]=="a"].reset_index()

In [69]:
num_transfers = get_ranking_transfer(accepted,"all").to_frame()
num_transfers["waivers"] = get_ranking_transfer(accepted,"w").to_frame()
num_transfers["free_agents"] = get_ranking_transfer(accepted,"f").to_frame()
num_transfers.loc["Dani","free_agents"] = 0

In [70]:
num_transfers.reset_index().rename(columns={'manager_id': 'manager'}).to_csv("findings/transfers/num_transfers.csv")

### Trade Winner & Loser


In [71]:
#get how long a transfer stays at the club, and how long until someone signs him up again
for indx, row in accepted.iterrows():
    player_in = row["players_id_in"]
    player_out = row["players_id_out"]

    for i in range(indx+1, len(accepted)):
        if player_out == accepted.loc[i, "players_id_in"]:
            accepted.loc[indx, "gw_until_in"] = accepted.loc[i, "gameweek"]-accepted.loc[indx, "gameweek"] #gameweeks until someone else signs this player
            accepted.loc[i, "past_manager"] = accepted.loc[indx, "manager_id"] #who owned him before
            accepted.loc[indx, "next_manager"] = accepted.loc[i, "manager_id"] #who will own him next
            break
    
    for i in range(indx+1, len(accepted)):
        if player_in == accepted.loc[i, "players_id_out"]:
            accepted.loc[indx, "gw_until_out"] = accepted.loc[i, "gameweek"]-accepted.loc[indx, "gameweek"] #gameweeks until this player leaves the team
            break
        

In [72]:
def get_points(player_id,gw):
    return data[(data["player_id"]==player_id)&(data["gameweek"]==gw)]["points"].iloc[0]

In [73]:
for indx, row in accepted.iterrows():
    gw = row["gameweek"]
    gws_out = row["gw_until_out"]
    player_in = row["players_id_in"]
    player_out = row["players_id_out"]

    points_in = 0
    points_out = 0

    
    if gws_out >= 3:
        for i in range(1,4):
            x = i - 1
            points_in_gw = get_points(player_in,gw + x)
            points_out_gw = get_points(player_out,gw + x)

            points_in += points_in_gw
            points_out += points_out_gw
    elif math.isnan(gws_out) == False:
        for i in range(1,int(gws_out+1)):
            x = i - 1
            points_in_gw = get_points(player_in,gw + x)
            points_out_gw = get_points(player_out,gw + x)

            points_in += points_in_gw
            points_out += points_out_gw

    accepted.loc[indx,"in_points"] = points_in
    accepted.loc[indx,"out_points"] = points_out
    accepted.loc[indx,"net_points"] = points_in - points_out

In [74]:
new_order = ['index', 'players_id_in', 'player_name_in', 'photo_in','team_in', 'in_points', 'past_manager' , 'gw_until_out', 
            'manager_id', 'gameweek', 'type', 'result', 'transfer_id', 'players_id_out',
            'player_name_out', 'team_out', 'out_points', "next_manager", 'gw_until_in', 'net_points']
accepted = accepted.reindex(columns=new_order)
accepted

,index,players_id_in,player_name_in,photo_in,team_in,in_points,past_manager,gw_until_out,manager_id,gameweek,type,result,transfer_id,players_id_out,player_name_out,team_out,out_points,next_manager,gw_until_in,net_points
0,0,189,Gray,172632,EVE,2.0,NaN,1.0,Ali,5,f,a,8830907,64,Lerma,BOU,3.0,NaN,NaN,-1.0
1,1,346,Andreas,156689,FUL,0.0,NaN,NaN,Yahya,5,f,a,8833951,465,Bowen,WHU,0.0,Santi,1.0,0.0
2,2,134,Kanté,116594,CHE,0.0,NaN,1.0,Ruslan,5,f,a,8834214,259,Tielemans,LEI,2.0,Ali,4.0,-2.0
3,3,358,Burn,78916,NEW,0.0,NaN,NaN,Yahya,5,f,a,8838057,43,Cash,AVL,0.0,Santi,22.0,0.0
4,4,342,Dalot,216051,MUN,10.0,NaN,23.0,Sami,5,f,a,8857423,504,Malacia,MUN,7.0,Ali,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,536,304,Sterling,103955,CHE,0.0,Sami,NaN,Ruslan,37,w,a,52819096,373,Willock,NEW,0.0,NaN,NaN,0.0
351,537,534,McNeil,433154,EVE,0.0,NaN,NaN,Santi,37,f,a,53216206,365,Murphy,NEW,0.0,NaN,NaN,0.0
352,538,156,Ward,55494,CRY,0.0,NaN,NaN,Santi,37,f,a,53216849,109,Veltman,BHA,0.0,NaN,NaN,0.0
353,540,504,Malacia,222690,MUN,0.0,Sami,NaN,Sami,38,w,a,53806324,8,Tierney,ARS,0.0,NaN,NaN,0.0


In [75]:
def get_transfer(data):
    filtering = data[["gameweek",'manager_id',"net_points","player_name_in","photo_in","team_in","player_name_out","team_out","in_points","out_points","gw_until_out"]]
    return filtering.sort_values(by=["net_points", 'manager_id'], ascending=[False, True])

In [76]:
transfers_net_points = get_transfer(accepted)

In [77]:
transfers_net_points.rename(columns={"manager_id":"manager","player_name_in":"player_name","photo_in":"photo"}).to_csv("findings/transfers/transfers_net_points.csv")

### Most times traded in

In [78]:
most_in = transfers_net_points.groupby(["manager_id","player_name_in","photo_in"]).size().sort_values(ascending=False).to_frame()

In [79]:
most_in

0
manager_id player_name_in   photo_in   
Ali        Tarkowski        17761     4
Santi      Veltman          111478    3
Ali        Bailey           215711    3
Sami       James            225796    3
Ali        Pickford         111234    3
...                                  ..
Ruslan     Xhaka            84450     1
Sami       Alexander-Arnold 169187    1
           Antony           467169    1
           Benrahma         172841    1
Youssef    Álvarez          461358    1

[307 rows x 1 columns]

In [80]:
most_in.reset_index().rename(columns={"manager_id":"manager","player_name_in":"player_name","photo_in":"photo",0:"player_id"}).to_csv("findings/transfers/most_in.csv")